In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

Populating the interactive namespace from numpy and matplotlib


/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)


## Alignment matrices for final production

Cancer matrices

In [2]:
def create_clean_matrices(onco_in, ts_in, ind_in, mut_out, aff_out):
    patient_mutations = pd.concat([pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/oncogenes.patient_mutation.csv', index_col=0).transpose(),
                                pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/tsgenes.patient_mutation.csv', index_col=0).transpose(),
                                pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/indels.patient_mutation.csv', index_col=0).transpose()]).transpose()
    patient_affinites = pd.concat([pd.read_csv(onco_in, index_col=0),
                                  pd.read_csv(ts_in, index_col=0),
                                  pd.read_csv(ind_in, index_col=0)]).transpose()
    print patient_affinites.shape, patient_mutations.shape
    # restrict to overlapping patients
    patients = [x for x in list(patient_affinites.index) if x in list(patient_mutations.index)]
    mutations = [x for x in list(patient_affinites.columns) if x in list(patient_mutations.columns)]
    patient_affinites = patient_affinites.ix[patients, mutations]
    patient_mutations = patient_mutations.ix[patients, mutations]
    print patient_affinites.shape, patient_mutations.shape
    # restrict to mutations >= 3 occurrences
    mutations = list(patient_mutations.sum()[patient_mutations.sum() > 2].index)
    print len(mutations)
    patient_affinites = patient_affinites.ix[:, mutations]
    patient_mutations = patient_mutations.ix[:, mutations]
    print patient_affinites.shape, patient_mutations.shape
    patient_affinites.to_csv(mut_out)
    patient_mutations.to_csv(aff_out)

In [3]:
for gene in ['DR', 'DP', 'DQ', 'all']:
    print gene
    create_clean_matrices('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/oncogenes.{0}.csv'.format(gene),
                          '/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/tsgenes.{0}.csv'.format(gene),
                          '/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/indels.{0}.csv'.format(gene),
                          '/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.cancer.{0}.csv'.format(gene),
                          '/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_mutations.cancer.{0}.csv'.format(gene))

DR
(7842, 1263) (10177, 1585)
(7656, 1261) (7656, 1261)
774
(7656, 774) (7656, 774)
DP
(7745, 1263) (10177, 1585)
(7579, 1261) (7579, 1261)
712
(7579, 712) (7579, 712)
DQ
(8097, 1263) (10177, 1585)
(7916, 1261) (7916, 1261)
773
(7916, 773) (7916, 773)
all
(7290, 1263) (10177, 1585)
(7137, 1261) (7137, 1261)
673
(7137, 673) (7137, 673)


In [9]:
gene = 'all'
df = pd.concat([pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/oncogenes.{0}.alternate.csv'.format(gene), index_col=0),
               pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/tsgenes.{0}.alternate.csv'.format(gene), index_col=0),
               pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/indels.{0}.alternate.csv'.format(gene), index_col=0)]).transpose()
df.to_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.cancer.{0}.alternate.csv'.format(gene))                  

Passengers

In [44]:
fasta_mutations = [x[1:].strip() for x in open('/cellar/users/ramarty/Data/hla_ii/presentation/fasta_files/passenger.fa').readlines() if '>' in x]
len(fasta_mutations)

1000

In [45]:
category = 'passenger'
mutations = [x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/presentation/residues/{0}.txt'.format(category)).readlines()]

In [46]:
patient_mutations = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/passenger.patient_mutation.csv', index_col=0)
patient_affinites = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/passenger.all.csv', index_col=0).transpose()

In [47]:
patients = [x for x in list(patient_affinites.index) if x in list(patient_mutations.index)]
mutations = [x for x in list(patient_affinites.columns) if x in list(patient_mutations.columns)]
patient_affinites = patient_affinites.ix[patients, mutations]
patient_mutations = patient_mutations.ix[patients, mutations]

In [48]:
patient_affinites.shape, patient_mutations.shape

((7137, 1000), (7137, 1000))

In [49]:
patient_mutations.to_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_mutations.cancer.passenger.csv')
patient_affinites.to_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.cancer.passenger.csv')

Germline

In [39]:
patient_mutations = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/germline.patient_mutations.csv', index_col=0)
patient_affinites = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/germline.all.csv', index_col=0).transpose()

In [40]:
patients = [x for x in list(patient_affinites.index) if x in list(patient_mutations.index)]
mutations = [x for x in list(patient_affinites.columns) if x in list(patient_mutations.columns)]
patient_affinites = patient_affinites.ix[patients, mutations]
patient_mutations = patient_mutations.ix[patients, mutations]

In [41]:
patient_affinites.shape, patient_mutations.shape

((2666, 1000), (2666, 1000))

In [43]:
patient_mutations.to_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_mutations.germline.all.csv')
patient_affinites.to_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.germline.all.csv')